In [1]:
# i am importing numpy and pands for working with data
import numpy as np
import pandas as pd

# for visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# for preprocessing
from pathlib import Path
from sklearn.model_selection import train_test_split

# i'll get my model up and running with tensorflow
import tensorflow as tf

# evaluating the performance using sklearn
from sklearn.metrics import confusion_matrix, classification_report